In [ ]:
import jupyanno as ja
import numpy as np

In [ ]:
%%javascript
require.undef('cornerstone');
require.undef('cornerstoneMath');
require.undef('cornerstoneTools');
require.config({
  paths: {
      'cornerstone-core': '//unpkg.com/cornerstone-core@2.2.4/dist/cornerstone.min',
      cornerstoneMath: '//unpkg.com/cornerstone-math@0.1.6/dist/cornerstoneMath.min',
      cornerstoneTools: '//unpkg.com/cornerstone-tools@2.3.9/dist/cornerstoneTools.min'
  }
});

In [ ]:
%%javascript
require(['cornerstone-core', 'cornerstoneTools', 'cornerstoneMath'], 
        function(cornerstone, cornerstoneTools, cornerstoneMath){
  $("#cs-viewer").remove();
    console.log('Loading cs:'+cornerstone);
      element.append("<div id='cs-viewer'></div>");
      $("#cs-viewer").width("512px");
      $("#cs-viewer").height("600px"); 
     var cs_viewer = document.getElementById('cs-viewer');
    (function (cs) {
    "use strict";
    function str2ab(str) {
        var buf = new ArrayBuffer(str.length*2); // 2 bytes for each char
        var bufView = new Uint16Array(buf);
        var index = 0;
        for (var i=0, strLen=str.length; i<strLen; i+=2) {
            var lower = str.charCodeAt(i);
            var upper = str.charCodeAt(i+1);
            bufView[index] = lower + (upper <<8);
            index++;
        }
        return bufView;
    }

    function parsePixelData(base64PixelData)
    {
        var pixelDataAsString = window.atob(base64PixelData);
        var pixelData = str2ab(pixelDataAsString);
        return pixelData;
    }
    var imageB64Data = "AQAAAAAAAAABAAAAAAAAAAEA"
    var imagePixelData = parsePixelData(imageB64Data);

    function loadPythonImage(imageId) {
        console.log('Reading Image:'+imageId);
        function getPixelData() {
            return imagePixelData;
        }
        var image = {
            imageId: imageId,
            minPixelValue: 0.0,
            maxPixelValue: 1.0,
            slope: 1.0,
            intercept: 0.0,
            windowCenter : 0.3333333333333333,
            windowWidth : 0.4714045207910317,
            getPixelData: getPixelData,
            rows: 3,
            columns: 3,
            height: 3,
            width: 3,
            color: false,
            columnPixelSpacing: .8984375,
            rowPixelSpacing: .8984375,
            sizeInBytes: 3 * 3 * 2
        };
        var out_promise=new Promise((resolve) => {
              resolve(image);
            });
        return {
            promise: out_promise,
            cancelFn: undefined
        };
    }
    // register our imageLoader plugin with cornerstone
    cs.registerImageLoader('numpy', loadPythonImage);
}(cornerstone));
    cornerstone.enable(cs_viewer);
    cornerstone.loadAndCacheImage('numpy://0').then(function(image) {
          cornerstone.displayImage(cs_viewer, image);
            // Enable our tools
          cornerstoneTools.mouseInput.enable(cs_viewer);
          cornerstoneTools.mouseWheelInput.enable(cs_viewer);
          cornerstoneTools.wwwc.activate(cs_viewer, 1); // Left Click
          cornerstoneTools.pan.activate(cs_viewer, 2); // Middle Click
          cornerstoneTools.zoom.activate(cs_viewer, 4); // Right Click
          cornerstoneTools.zoomWheel.activate(cs_viewer); // Mouse Wheel
    })
})

In [ ]:
from IPython.display import Javascript
import jupyanno as ja, numpy as np
simple_cs_tools = """
require.undef('cornerstone');
require.undef('cornerstoneMath');
require.undef('cornerstoneTools');
require.config({
  paths: {
      'cornerstone-core': '//unpkg.com/cornerstone-core@2.2.4/dist/cornerstone.min',
      cornerstoneMath: '//unpkg.com/cornerstone-math@0.1.6/dist/cornerstoneMath.min',
      cornerstoneTools: '//unpkg.com/cornerstone-tools@2.3.9/dist/cornerstoneTools.min'
  }
});
require(['cornerstone-core', 'cornerstoneTools', 'cornerstoneMath'], 
    function(cornerstone, cornerstoneTools, cornerstoneMath){
    $("#cs-viewer").remove();
    console.log('Loading cs:'+cornerstone+','+cornerstoneMath+','+cornerstoneTools);
    element.append("<div id='{item_name}'></div>");
    $("#{item_name}").width("512px");
    $("#{item_name}").height("600px"); 
    var cs_viewer = document.getElementById('{item_name}');
    {numpy_loader_js}
    cornerstone.enable(cs_viewer)
    cornerstone.loadAndCacheImage('numpy://0').then(function(image) {
        cornerstone.displayImage(cs_viewer, image);
        // Enable our tools
        cornerstoneTools.mouseInput.enable(cs_viewer);
        cornerstoneTools.mouseWheelInput.enable(cs_viewer);
        cornerstoneTools.wwwc.activate(cs_viewer, 1); // Left Click
        cornerstoneTools.pan.activate(cs_viewer, 2); // Middle Click
        cornerstoneTools.zoom.activate(cs_viewer, 4); // Right Click
        cornerstoneTools.zoomWheel.activate(cs_viewer); // Mouse Wheel
    })
})
"""
Javascript(ja.utils.fancy_format(simple_cs_tools,
                                 item_name = 'big_dicom',
                     numpy_loader_js = ja.cornerstone.make_pyloader_js(np.eye(256))))


In [ ]:
from traitlets import Unicode, validate, Bytes
import ipywidgets as widgets

In [ ]:
class CornerstoneWidget(widgets.DOMWidget):
    _view_name = Unicode('CornerstoneWidget').tag(sync=True)
    _view_module = Unicode('cs_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    title_field = Unicode('Awesome Widget').tag(sync=True)
    img_bytes = Bytes(b'').tag(sync=True)

In [ ]:
%%javascript
require.undef('cs_widget');
define('cs_widget', ["@jupyter-widgets/base", "cornerstone"], function(widgets, cs) {

    var CornerstoneWidget = widgets.DOMWidgetView.extend({

        render: function() {
            console.log(cs)
            //cs.enable(this.el);
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },

        value_changed: function() {
            this.el.textContent = this.model.get('title_field');
        },
    });

    return {
        CornerstoneWidget : CornerstoneWidget
    };
});

In [ ]:
w = CornerstoneWidget()
w

In [ ]:
w.title_field = 'Hai'